# Avaliação de Fairness no modelo

Para a avaliação de fairness, será utilizada a métrica Equal opportunity, e será realizada a análise na classe "sexo"

Primeiramente, vamos calcular a métrica de fairness para a classe sexo, analisar, e utilizar técnicas de mitigação de viés a fim de comparar os resultados com o modelo anterior

In [33]:
# Carregando os dados de teste
import pandas as pd
test_data = pd.read_csv('entrega-1/df_teste.csv')
print(test_data.columns)


Index(['Quantidade ovos encontrados', 'Escolaridade', 'Sexo', 'Resultado',
       'Exame_Desconhecido', 'Exame_Nao_Realizado', 'Exame_Negativo',
       'Exame_Positivo', 'Tratamento_Ignorado', 'Tratamento_Nao',
       'Tratamento_Sim_Oxaminiquine', 'Tratamento_Sim_Praziquantel',
       'Forma_Aguda', 'Forma_Hepato_Esplenica', 'Forma_Hepato_Intestinal',
       'Forma_Intestinal', 'Forma_Nao_especificado', 'Forma_Outra', 'Idade',
       'Regiao_Centro-Oeste', 'Regiao_Nao_especificado', 'Regiao_Nordeste',
       'Regiao_Norte', 'Regiao_Sudeste', 'Regiao_Sul'],
      dtype='object')


In [34]:
# Removendo colunas que não agregam na avaliação (Assim como realizado no treinamento)
test_data = test_data.drop(columns=['Exame_Desconhecido', 'Tratamento_Ignorado', 'Forma_Nao_especificado', 'Regiao_Nao_especificado'])

In [35]:
# Dividindo os conjuntos de predições e target
X_test = test_data.drop(columns=['Resultado'])
y_test = test_data['Resultado']
sexo = test_data['Sexo']
escolaridade = test_data['Escolaridade']

In [36]:
# Carregando o modelo
import pickle
with open('source/models/best_model.pkl', 'rb') as file:
    best_model = pickle.load(file)


Realizando as predições no conjunto de teste para o melhor modelo encontrado: 

In [37]:
X_test.columns

Index(['Quantidade ovos encontrados', 'Escolaridade', 'Sexo',
       'Exame_Nao_Realizado', 'Exame_Negativo', 'Exame_Positivo',
       'Tratamento_Nao', 'Tratamento_Sim_Oxaminiquine',
       'Tratamento_Sim_Praziquantel', 'Forma_Aguda', 'Forma_Hepato_Esplenica',
       'Forma_Hepato_Intestinal', 'Forma_Intestinal', 'Forma_Outra', 'Idade',
       'Regiao_Centro-Oeste', 'Regiao_Nordeste', 'Regiao_Norte',
       'Regiao_Sudeste', 'Regiao_Sul'],
      dtype='object')

In [38]:
y_pred = best_model.predict(X_test)

print(sexo.unique())

[1 0]


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


No nosso caso, o valor 0 é masculino e 0 é feminino

In [39]:
from sklearn.metrics import recall_score

grupos = sexo.unique()
labels = ['Masculino', 'Feminino']
for value in grupos:
    idx_gp = sexo == value # Escolhe o sexo definido e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred[idx_gp])
    print(f'Recall para o grupo {labels[value]}: {recall:.4f}')

Recall para o grupo Feminino: 0.7000
Recall para o grupo Masculino: 0.7600


Testando para escolaridade

In [41]:
escolaridade_labels = {
    0: "Analfabeto",
    1: "1ª a 4ª série incompleta",
    2: "4ª série completa",
    3: "5ª a 8ª série incompleta",
    4: "Ensino fundamental completo",
    5: "Ensino médio incompleto",
    6: "Ensino médio completo",
    7: "Educação superior incompleta",
    8: "Educação superior completa",
}


for value in escolaridade_labels.keys():
    idx_gp = escolaridade == value # Escolhe a escolaridade definida e calcula o recall
    recall = recall_score(y_test[idx_gp], y_pred[idx_gp])
    print(f'Recall para o grupo {escolaridade_labels[value]}: {recall:.4f}')

Recall para o grupo Analfabeto: 0.7636
Recall para o grupo 1ª a 4ª série incompleta: 0.7500
Recall para o grupo 4ª série completa: 1.0000
Recall para o grupo 5ª a 8ª série incompleta: 0.5000
Recall para o grupo Ensino fundamental completo: 0.6250
Recall para o grupo Ensino médio incompleto: 1.0000
Recall para o grupo Ensino médio completo: 0.0000
Recall para o grupo Educação superior incompleta: 0.0000
Recall para o grupo Educação superior completa: 0.0000


/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lipecorradini/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Como não temos representantes de todas as classes para graus mais altos de educação nos dois conjuntos alvo (óbito e sobrevivência), torna-se inviável de avaliar tal métrica